In [1]:
import pandas as pd
import numpy as np
import unidecode

from datetime import date, datetime, timedelta

import funcoes_eda as ufm


In [2]:
def check_null(df):
    """ verifica se há valores ausentes em cada coluna do geral.
    INPUT: df - o geral analisado
    OUTPUT: imprime o resultado da verificação, que pode ser:
    "Valores ausentes de {}: {}", com o nome da coluna e sua quantidade de valores ausentes ou
    "Não existem colunas com valores ausentes" 
    """
    if df.isnull().any().sum() != 0:
        for i in df.columns[df.isnull().any()].values.tolist():
            print("Valores ausentes de {}: {}".format(i,df[i].isnull().sum()))
    else:
        print("Não existem colunas com valores ausentes")

def normaliza_txt_upper(series):
    return [unidecode.unidecode(x).upper() for x in series]

def municipios_brio(uf):
    df_municipios_uf = pd.read_csv("https://raw.githubusercontent.com/turicas/covid19-br/master/covid19br/data/populacao-por-municipio-2020.csv")
    # ("https://raw.githubusercontent.com/turicas/covid19-br/master/data/populacao-estimada-2019.csv")
#     ../populacao-estimada-2019.csv
    df_municipios_uf = df_municipios_uf[df_municipios_uf.state == uf][['city']].reset_index(drop=True)
    add_rows = pd.DataFrame(['TOTAL NO ESTADO', 'Importados/Indefinidos'], columns=['city'])
    df_municipios_uf = pd.concat([add_rows, df_municipios_uf], ignore_index=True)
    df_municipios_uf['municipio'] = normaliza_txt_upper(df_municipios_uf['city'])
    return df_municipios_uf


## Crawler RO

#### [Link para TESTE RO: 21/05/2021](http://www.rondonia.ro.gov.br/edicao-427-boletim-diario-sobre-coronavirus-em-rondonia/)


In [10]:
dias_para_tras = 0

reference_day = datetime.strptime("2021-05-20", "%Y-%m-%d").date()
print(f"reference_day: {reference_day}")
reference_boletim_no = 426
print(f"reference_boletim_no: {reference_boletim_no}")

today = date.today() 
print(f"today: {today}")
today_boletim_no = reference_boletim_no + (today - reference_day).days - dias_para_tras
# # para pegar o de ontem, rodar esta linha
# today_boletim_no = reference_boletim_no + (today - reference_day).days -1
print(f"today_boletim_no: {today_boletim_no}")

yesterday = today - timedelta(days=1)
print(f"yesterday: {yesterday}")

now = str((datetime.now().strftime('%Y%m%d%H%M')))
print(f"now: {now}")

str_date = str(((today - timedelta(days=dias_para_tras)).strftime('%d-%m-%Y')))
# # para pegar o de ontem, rodar esta linha
# str_date = str((yesterday.strftime('%d-%m-%Y')))
# str_date = "20210406"
print(f"str_date: {str_date}")

path_date = str((today.strftime('%Y/%m')))
print(f"path_date: {path_date}")

uf = 'RO'
print(f"state: {uf}")

capital = 'Porto Velho'
print(f"capital: {capital}\n")


reference_day: 2021-05-20
reference_boletim_no: 426
today: 2021-06-03
today_boletim_no: 440
yesterday: 2021-06-02
now: 202106031952
str_date: 03-06-2021
path_date: 2021/06
state: RO
capital: Porto Velho



In [11]:
# define url de origem do boletim
url = f'http://www.rondonia.ro.gov.br/edicao-{today_boletim_no}-boletim-diario-sobre-coronavirus-em-rondonia/'
print(f"URL do boletim: {url}\n")


URL do boletim: http://www.rondonia.ro.gov.br/edicao-440-boletim-diario-sobre-coronavirus-em-rondonia/



In [6]:
dfs = pd.read_html(url)
print(len(dfs))

# for table in dfs:
#     display(table.head())
    
# dfs[0]

6


In [7]:
df_ro = dfs[0].copy()
df_ro.columns = df_ro.iloc[1]
df_ro.rename_axis(None, axis=1, inplace=True)
df_ro.drop(df_ro.index[0:2], inplace=True)
df_ro = df_ro.reset_index(drop=True)
df_ro.rename(columns=lambda x: unidecode.unidecode(x).strip().replace(" ", "_").replace("\n", "").replace("*", "").lower(), inplace=True)
df_ro.rename(columns={'casos_totais': 'confirmados', 'obitos_totais': 'mortes'}, inplace=True)
df_ro = df_ro[['municipio', 'confirmados', 'mortes']]
df_ro['confirmados'] = df_ro['confirmados'].str.replace(".", "").astype(int)
df_ro['mortes'] = df_ro['mortes'].str.replace(".", "").astype(int)
check_null(df_ro)

df_ro


Não existem colunas com valores ausentes


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  if __name__ == '__main__':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  # Remove the CWD from sys.path while we load stuff.


,municipio,confirmados,mortes
0,Porto Velho,77749,2367
1,Ariquemes,20181,439
2,Ji-Paraná,16980,502
3,Cacoal,12392,250
4,Vilhena,12019,232
5,Jaru,7060,160
6,Machadinho D’Oeste,6053,90
7,Rolim de Moura,6029,151
8,Pimenta Bueno,5601,83
9,Guajará-Mirim,5325,210


In [8]:
# municipios_brio(uf)

In [9]:
df = df_ro.copy()
display(df.query("municipio == @capital"))

df = df[df.municipio != "Total geral"]

print(f"\nsoma confirmados: {df.confirmados.sum()}\nsoma mortes: {df.mortes.sum()}")
# df = df.replace({'Ignorado':'Importados/Indefinidos', 'Total': 'TOTAL NO ESTADO'})
df = df.groupby(['municipio'], as_index=False).sum()

# # Substitui nome original de município, se necessário 
# df['municipio'] = df['municipio'].str.replace('M. Thaumaturgo', 'Marechal Thaumaturgo')

df['municipio'] = normaliza_txt_upper(df['municipio'])
display(df.query("municipio == @capital.upper()"))

df = pd.merge(municipios_brio(uf), df, on=['municipio'], how='outer')

print(f"\nCONFLITOS DE GRAFIA DE MUNICÍPIO:\n\n{df[df['city'].isnull()]}" if df['city'].isnull().any() else "\nNENHUM CONFLITOS DE GRAFIA DE MUNICÍPIO!")

df.drop(columns=['municipio'], inplace=True)
df.rename(columns={'city': 'municipio'}, inplace=True)

df.at[0, 'confirmados'] = df.confirmados.sum()
df.at[0, 'mortes'] = df.mortes.sum()

# df.query("confirmados != 0 & mortes != 0")
df[['confirmados', 'mortes']] = df[['confirmados', 'mortes']].astype("Int64")

print(f"\n{df.shape[0]} linhas e {df.shape[1]} colunas.\n")
check_null(df)

# só usar se for mesmo para preencher com zero neste estado
df = df.fillna(0)

# SALVA ARQUIVO
now = str((datetime.now().strftime('%Y%m%d%H%M')))
filename = f'caso-{uf}-{str_date}_csv_ses.csv' 
df.to_csv(filename, index=False)
print(f"\nArquivo salvo: {filename}")

display(df)

,municipio,confirmados,mortes
0,Porto Velho,77749,2367



soma confirmados: 231544
soma mortes: 5792


,municipio,confirmados,mortes
36,PORTO VELHO,77749,2367



NENHUM CONFLITOS DE GRAFIA DE MUNICÍPIO!

54 linhas e 3 colunas.

Valores ausentes de confirmados: 1
Valores ausentes de mortes: 1

Arquivo salvo: caso-RO-02-06-2021_csv_ses.csv


,municipio,confirmados,mortes
0,TOTAL NO ESTADO,231544,5792
1,Importados/Indefinidos,0,0
2,Alta Floresta D'Oeste,3731,63
3,Alto Alegre dos Parecis,1401,41
4,Alto Paraíso,1756,43
5,Alvorada D'Oeste,1011,25
6,Ariquemes,20181,439
7,Buritis,4425,62
8,Cabixi,710,18
9,Cacaulândia,791,10
